In [1]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import hashlib
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.hashes import SHA256
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

In [3]:
# Fungsi untuk menghasilkan kunci dari password menggunakan PBKDF2HMAC
def derive_key_from_password(password: str, salt: bytes):
    kdf = PBKDF2HMAC(
        algorithm=SHA256(),
        length=32,  # AES-256 membutuhkan kunci 32 byte
        salt=salt,
        iterations=100000  # Iterasi tinggi untuk keamanan lebih baik
    )
    return kdf.derive(password.encode())


In [4]:

# Fungsi untuk mengenkripsi file [yang dikirim = salt, iv, dan hasil keynya]
def encrypt_file(input_file, output_file, password):
    salt = os.urandom(16)  # Menghasilkan salt acak
    iv = os.urandom(12)  # AES-GCM membutuhkan IV 12 byte
    
    key = derive_key_from_password(password, salt)  # Kunci dari password
    aesgcm = AESGCM(key)  # Inisialisasi AES-GCM

    # Membaca file dalam mode binary
    with open(input_file, 'rb') as f:
        plaintext = f.read()

    # Enkripsi dengan AES-GCM
    ciphertext = aesgcm.encrypt(iv, plaintext, None)

    # Menyimpan salt + IV + ciphertext ke dalam file output
    with open(output_file, 'wb') as f:
        f.write(salt+ iv + ciphertext)

    print(f"File '{input_file}' berhasil dienkripsi sebagai '{output_file}'.")
    print(f"key: {key.hex()}")
    return key.hex()



In [5]:
# Contoh penggunaan
password = 'mypassword123'
key = encrypt_file('rahasia.txt', 'document_encrypted.bin', password)

File 'rahasia.txt' berhasil dienkripsi sebagai 'document_encrypted.bin'.
key: b568fc05ef564d0a6c2678b880ec91e551f051445cb24c01b9be4b4ec410e8c7


In [6]:
# Fungsi untuk mendekripsi file
def decrypt_file(input_file, output_file, key_hex):
    # Membaca file terenkripsi
    with open(input_file, 'rb') as f:
        encrypted_data = f.read()
        
    # Inisialisasi AES-GCM dengan key dan IV yang sama
    salt = encrypted_data[0:16]
    iv = encrypted_data[16:28]
    ciphertext= encrypted_data[28:]

    key = derive_key_from_password(password, salt)
    aesgcm = AESGCM(key)

    # Dekripsi data
    plaintext = aesgcm.decrypt(iv, ciphertext, None)

    # Menyimpan hasil dekripsi ke file
    with open(output_file, 'wb') as f:
        f.write(plaintext)

    print(f"File '{input_file}' berhasil didekripsi sebagai '{output_file}'.")



In [7]:
# Contoh penggunaan: Mendekripsi file Word atau Excel
decrypt_file('document_encrypted.bin', 'document_decrypted.txt',key)

File 'document_encrypted.bin' berhasil didekripsi sebagai 'document_decrypted.txt'.


In [8]:
# Contoh penggunaan: Mendekripsi file Word atau Excel
decrypt_file('document_encrypted_albern', 'document_decrypted_albern.txt',key)

File 'document_encrypted_albern' berhasil didekripsi sebagai 'document_decrypted_albern.txt'.


In [11]:
password = 'kenrikyoo123'
key = encrypt_file('rahasia.txt', 'document_encrypted.bin', password)

decrypt_file('document_encrypted_alfandi', 'document_decrypted_alfandi.txt', key)

File 'rahasia.txt' berhasil dienkripsi sebagai 'document_encrypted.bin'.
key: 9ae1056da0da4efc1aa56cc07df8f2a919fa78f8151686d308428194863d6f2d
File 'document_encrypted_alfandi' berhasil didekripsi sebagai 'document_decrypted_alfandi.txt'.


In [12]:
password = 'secretpassword123'
key = encrypt_file('message.txt', 'document_encrypted.bin', password)

File 'message.txt' berhasil dienkripsi sebagai 'document_encrypted.bin'.
key: 2af2779cfa1913748b262ba899557ebb1b7259fc0fdeffa5bb898ecbff692095
